# Import Libraries and Download NLTK Resources

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#  Define Text Preprocessing Functions

In [2]:
# Function to preprocess text
def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    tokens = word_tokenize(text)  # Tokenize
    tokens = [word for word in tokens if word.isalpha()]  # Remove non-alphabetic tokens
    stop_words = set(stopwords.words('english'))  # Stopwords
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    return tokens


# Define Vocabulary Building Function

In [3]:
# Function to build vocabulary
def build_vocabulary(data):
    vocabulary = defaultdict(int)
    for text in data:
        for word in text:
            vocabulary[word] += 1
    return {word: index for index, word in enumerate(vocabulary) if vocabulary[word] >= 5}


# Define Naive Bayes Classifier Training Function

In [4]:
# Function to train Naive Bayes Classifier
def train_naive_bayes(data, vocab):
    word_counts = {class_: defaultdict(int) for class_ in np.unique(data['generated'])}
    class_counts = defaultdict(int)

    for _, row in data.iterrows():
        label = row['generated']
        class_counts[label] += 1
        for word in row['processed_text']:
            if word in vocab:
                word_counts[label][word] += 1

    total_docs = len(data)
    word_probs = {class_: {word: (word_counts[class_][word] + 1) / (class_counts[class_] + len(vocab))
                           for word in vocab}
                  for class_ in word_counts}
    class_probs = {class_: class_counts[class_] / total_docs for class_ in class_counts}

    return class_probs, word_probs


# Define Classification Function

In [5]:
# Function to classify new instances
def classify(text, class_probs, word_probs, vocab):
    text_words = set(preprocess_text(text))
    class_scores = {class_: np.log(class_prob) for class_, class_prob in class_probs.items()}
    for class_, word_prob in word_probs.items():
        for word in text_words:
            if word in vocab:
                class_scores[class_] += np.log(word_prob.get(word, 1 / (len(vocab) + sum(word_prob.values()))))

    return max(class_scores, key=class_scores.get)


# Load and Preprocess Data

In [11]:
# Load and preprocess the data
essays = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
prompts = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')
merged_data = pd.merge(essays, prompts, on='prompt_id')

# Preprocessing texts
merged_data['processed_text'] = merged_data['text'].apply(preprocess_text)

# Split Data and Build Vocabulary

In [12]:
# Splitting the data
train_data = merged_data.sample(frac=0.8, random_state=1)
dev_data = merged_data.drop(train_data.index)

# Building vocabulary from training data
vocab = build_vocabulary(train_data['processed_text'])

# Train Naive Bayes Classifier and Evaluate on Development Data

In [13]:
# Training the Naive Bayes Classifier
class_probs, word_probs = train_naive_bayes(train_data, vocab)

# Evaluating the classifier on development data
dev_data['predicted'] = dev_data['text'].apply(lambda text: classify(text, class_probs, word_probs, vocab))
accuracy = np.mean(dev_data['predicted'] == dev_data['generated'])
print(f'Accuracy on development set: {accuracy*100}%')

Accuracy on development set: 99.27536231884058%


# Probability of the word "the"

In [14]:
# Assuming vocab is already defined
word_of_interest = "the"

# Calculate the probability
probability_of_word = vocab.get(word_of_interest, 0) / len(vocab)

# Print the result
print(f'Probability of the word "{word_of_interest}": {probability_of_word}')

Probability of the word "the": 0.0


# Conditional Probability based on the class

In [15]:
llm_data = merged_data[merged_data['generated'] == 1]
p_the_llm = llm_data['processed_text'].apply(lambda text: "the" in text).mean()
print(f'Conditional probability of "the" given LLM: {p_the_llm}')


Conditional probability of "the" given LLM: 0.0


# Derive Top 10 words that predict each class

In [17]:
top_words_llm = sorted(vocab, key=lambda word: word_probs[1][word], reverse=True)[:10]
top_words_human = sorted(vocab, key=lambda word: word_probs[0][word], reverse=True)[:10]

print(f'Top 10 words predicting LLM: {top_words_llm}')
print(f'Top 10 words predicting Human: {top_words_human}')

Top 10 words predicting LLM: ['electoral', 'votes', 'college', 'state', 'vote', 'win', 'president', 'popular', 'candidate', 'majority']
Top 10 words predicting Human: ['electoral', 'people', 'car', 'college', 'vote', 'cars', 'states', 'would', 'president', 'usage']


# compare effect of smooting

In [ ]:
def train_naive_bayes(data, vocab, smoothing_method='laplace', smoothing_parameter=1):
    word_counts = {class_: defaultdict(int) for class_ in np.unique(data['generated'])}
    class_counts = defaultdict(int)

    for _, row in data.iterrows():
        label = row['generated']
        class_counts[label] += 1
        for word in row['processed_text']:
            if word in vocab:
                word_counts[label][word] += 1

    total_docs = len(data)
    word_probs = {class_: {} for class_ in word_counts}
    class_probs = {class_: class_counts[class_] / total_docs for class_ in class_counts}

    for class_, word_count in word_counts.items():
        for word in vocab:
            if smoothing_method == 'laplace':
                word_probs[class_][word] = (word_count.get(word, 0) + smoothing_parameter) / \
                                           (class_counts[class_] + smoothing_parameter * len(vocab))
            elif smoothing_method == 'add-k':
                word_probs[class_][word] = (word_count.get(word, 0) + smoothing_parameter) / \
                                           (class_counts[class_] + smoothing_parameter)
            else:
                raise ValueError('Invalid smoothing method')

    return class_probs, word_probs


In [19]:
# Train Naive Bayes with Laplace smoothing (add-one)
class_probs_laplace, word_probs_laplace = train_naive_bayes(train_data, vocab, smoothing_method='laplace', smoothing_parameter=1)

# Train Naive Bayes with add-k smoothing
class_probs_addk, word_probs_addk = train_naive_bayes(train_data, vocab, smoothing_method='add-k', smoothing_parameter=0.1)

# Evaluate classifiers with different smoothing
dev_data['predicted_laplace'] = dev_data['text'].apply(lambda text: classify(text, class_probs_laplace, word_probs_laplace, vocab))
dev_data['predicted_addk'] = dev_data['text'].apply(lambda text: classify(text, class_probs_addk, word_probs_addk, vocab))

accuracy_laplace = np.mean(dev_data['predicted_laplace'] == dev_data['generated'])
accuracy_addk = np.mean(dev_data['predicted_addk'] == dev_data['generated'])

print(f'Accuracy with Laplace smoothing: {accuracy_laplace*100}%')
print(f'Accuracy with add-k smoothing: {accuracy_addk*100}%')


Accuracy with Laplace smoothing: 99.27536231884058%
Accuracy with add-k smoothing: 95.65217391304348%
